In [127]:
import boto3, os
import json, sys
from pprint import pprint
from botocore.exceptions import NoCredentialsError, ClientError
import logging
from tqdm import tqdm
import pandas as pd, os


In [128]:
#  just change these values
sem_plus_sub = '07-css'  # no capital, no underscore
file_dir = r'C:\Users\Ramstein\Desktop\textract\s3_upload_test'

In [129]:
bucket_name = sem_plus_sub+'-textract-upload'
region = 'us-west-2'

# ACCESS_KEY = 'hadiugyaduyg'
# SECRET_KEY = 'hidauhuadyh'

files = os.listdir(file_dir)
s3_client = boto3.client('s3')


def create_bucket(bucket_name, region=None):
    # if Bucket is already owned, it will move forward.
    try:
        if region is None:
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

def upload_to_aws(local_file, bucket, s3_file_name):
    try:
        s3_client.upload_file(local_file, bucket, s3_file_name)
        return True
    except FileNotFoundError:
        print("File not found: "+local_file)
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False

In [130]:
# creating bucket
if create_bucket(bucket_name, region=region): print('Bucket:'+ bucket_name+ 'created.')

ERROR:root:An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [131]:
# uploading the png files
for file in tqdm(files):
    local_file = os.path.join(file_dir, file)
    s3_file_name=file
    upload_to_aws(local_file, bucket_name, s3_file_name)


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:06<00:00, 12.55s/it]


In [132]:
lines = []
# amazon s3
s3 = boto3.resource('s3')

# amazon textract
textract = boto3.client(service_name='textract', region_name=region)

txt_file_path = os.path.join(file_dir, sem_plus_sub)+'.txt'
csv_file_path = os.path.join(file_dir, sem_plus_sub)+'.csv'

txt_file_opened = open(txt_file_path, 'w')


In [133]:
# detecting text in the documents and writing to txt
for file in tqdm(files):
    try:
        response = textract.detect_document_text(
            Document={
                "S3Object": {
                    "Bucket": bucket_name,
                    "Name": file
                }
            }
        )
        for item in response["Blocks"]:
            if item["BlockType"] == "LINE":
#                 lines.append(item["Text"])
#                 print(item["Text"])
                txt_file_opened.write(item["Text"]+'\n')                

    except Exception as e:
        print(e)
        
txt_file_opened.close()


  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

An error occurred (UnsupportedDocumentException) when calling the DetectDocumentText operation: Request has unsupported document format



 17%|██████████████                                                                      | 1/6 [00:00<00:04,  1.01it/s]

An error occurred (UnsupportedDocumentException) when calling the DetectDocumentText operation: Request has unsupported document format


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:26<00:00,  4.67s/it]


In [134]:
def quanify():
    df = pd.DataFrame({"Question1": [''],
                       'Question2': [''],
                       'Answer': ['']
                       })
    with open(txt_file_path, 'r') as file:
        lines = file.readlines()

        sub_name = ['DESIGN OF STRUCTURE - III']
        contents = []
        # if a line conatains any of these, remove that
        drop_str = ['QUESTIONS-ANSWERS', 'LONG ANSWER TYPE AND MEDIUM ANSWER TYPE QUESTIONS',
                    'VERY IMPORTANT QUESTIONS', 'FOLLOWING QUESTIONS ARE VERY IMPORTANT. THESE QUESTIONS',
                    'MAY BE ASKED IN YOUR SESSIONALS AS WELL AS', 'UNIVERSITY EXAMINATION',
                    'MARKS', '2 MARKS QUESTIONS', 'QUESTION', 'PART-1', 'PART-2', 'PART-3', 'PART-4', 'PART-5',
                    'PART-6', 'PART-7', 'PART-8', 'PART-9', 'PART-10', 'PART-11', 'PART-12', 'PART-13', 'PART-14',
                    'PART-15', 'PART-16', 'PART-17', 'PART-18',
                    'PART-19', 'PART-20', 'PART-21', 'PART-22', 'PART-23', 'PART-24', 'PART-25', 'PART-26', 'PART-27',
                    'PART-28', 'PART-29', 'PART-30',
                    'AKTU', 'MARKS 01', 'MARKS 02', 'MARKS 05', 'MARKS 10', 'MARKS 15', '2010-11', '2011-12', '2012-13',
                    '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22',
                    '2022-23']
        sem = 'CE-SEM'

        question_end = ['.', '?', ',', '!']
        drop_after_qa_detn = ['QUE ', 'OR', 'ANSWER']  # Leading part of the Que line will also be removed.
        lines1 = []
        que_line_start1, que_line_end1, que_line_start2, que_line_end2, ans_line_start, ans_line_end = 0, 0, 0, 0, 0, 0
        que_found1, que_found2, ans_found = 0, 0, 0
        que_idx1, que_idx2, ans_idx = 0, 0, 0

        for line in lines:
            line = line.split('\n')[0].upper()
            lines1.append(line)

        # appending all the dropping strings to one list
        drop_str.append(sem)
        for w in sub_name: drop_str.append(w)
        for w in contents: drop_str.append(w)

        lines = []
        # removing all the sub-strings
        for line in lines1:
            found = 0
            for sub_str in drop_str:
                if line.find(sub_str) != -1: found = 1
            if found != 1: lines.append(line); found = 0
        lines1 = []

        for i, line in enumerate(lines):
            if line.find('QUE') != -1 or line.find('QVE') != -1:
                que_line_start1 = i
                ans_line_end = i - 1
                que_found1 = 1

            if line.find('OR') != -1 or line == 'OR':
                que_line_end1 = i-1
                que_line_start2 = i
                que_found2 = 1

            if line.find('ANSWER') != -1 or line == 'ANSWER':
                que_line_end2 = i - 1
                ans_line_start = i
                ans_found = 1

            if que_found1 == 1:
                answer = lines[ans_line_start + 1]
                if ans_line_start < ans_line_end:
                    for a in range(ans_line_end - ans_line_start - 1):
                        ans_line_start += 1
                        answer += ' ' + lines[ans_line_start + 1]
                que_found1 = 0
                # Writing to DataFrame
                df.at[ans_idx, 'Answer'] = answer
                ans_idx += 1

            if ans_found == 1:
                question = lines[que_line_start1 + 1]
                if que_line_start1 < que_line_end1:
                    for q in range(que_line_end1 - que_line_start1 - 1):
                        que_line_start1 += 1
                        question += ' ' + lines[que_line_start1 + 1]
                # ans_found = 0 # not set it here
                # Writing to DataFrame
                df.at[que_idx1 +1, 'Question1'] = question
                que_idx1 += 1

            if ans_found == 1:
                question = lines[que_line_start2 + 1]
                if que_line_start2 < que_line_end2:
                    for q in range(que_line_end2 - que_line_start2 - 1):
                        que_line_start2 += 1
                        question += ' ' + lines[que_line_start2 + 1]
                ans_found = 0
                # Writing to DataFrame
                df.at[que_idx2 +1, 'Question2'] = question;que_idx2 += 1

    return df

    # new_row = pd.DataFrame({"Question1": 'what is magnetic confinement?',
    #                         'Question2': 'Describe magnetic confinement?',
    #                         'Answer': 'sheilding the plasma with magnetic field lines.'
    #                         }, index=[1])
    # df = pd.concat([df, new_row]).reset_index(drop=True)

In [135]:
df = quanify()
df.head()

,Question1,Question2,Answer
0,,,AVEI:L
1,AVEI:L,AVEI:L 1.1. WHAT IS STEEL STRUCTURE ? WHAT ARE...,A STEEL STRUCTURE : A STEEL STRUCTURE IS AN AS...
2,WHAT ARE THE ADVANTAGES AND DISADVANTAGES OF S...,WRITE ADVANTAGES OF STEEL USED AS A STRUCTURAL...,A ADVANTAGES : FOLLOWING ARE THE ADVANTAGES OF...
3,WHAT IS STRUCTURAL STEEL AND WHAT ARE ITS PROP...,DESCRIBE SOME OF THE MECHANICAL PROPERTIES OF ...,STEEL : IT IS PROBABLY THE MOST VERSATILE AND ...
4,DISCUSS IN BRIEF A GENERAL CHEMICAL COMPOSITIO...,GIVE THE CHEMICAL COMPOSITION OF STRUCTURAL ST...,A. CHEMICAL COMPOSITION OF STEEL : THE CHEMICA...


In [136]:
df = df.drop([0]) # look only if it needs to drop the row then drop it
df.head()

,Question1,Question2,Answer
1,AVEI:L,AVEI:L 1.1. WHAT IS STEEL STRUCTURE ? WHAT ARE...,A STEEL STRUCTURE : A STEEL STRUCTURE IS AN AS...
2,WHAT ARE THE ADVANTAGES AND DISADVANTAGES OF S...,WRITE ADVANTAGES OF STEEL USED AS A STRUCTURAL...,A ADVANTAGES : FOLLOWING ARE THE ADVANTAGES OF...
3,WHAT IS STRUCTURAL STEEL AND WHAT ARE ITS PROP...,DESCRIBE SOME OF THE MECHANICAL PROPERTIES OF ...,STEEL : IT IS PROBABLY THE MOST VERSATILE AND ...
4,DISCUSS IN BRIEF A GENERAL CHEMICAL COMPOSITIO...,GIVE THE CHEMICAL COMPOSITION OF STRUCTURAL ST...,A. CHEMICAL COMPOSITION OF STEEL : THE CHEMICA...
5,DRAW A STRESS STRAIN CURVE FOR MILD STEEL AND ...,DISCUSS STRESS STRAIN BEHAVIOUR OF MILD STEEL.,NaN


In [137]:
df.to_csv(csv_file_path, index_label='Id')
print('QA csv file created: '+ csv_file_path)

QA csv file created: C:\Users\Ramstein\Desktop\textract\s3_upload_test\07-css.csv


In [140]:
# uploading csv file to the same bucket

if upload_to_aws(csv_file_path, bucket_name, sem_plus_sub+'.csv'):
    print(csv_file_path + ' uploaded to s3.')


C:\Users\Ramstein\Desktop\textract\s3_upload_test\07-css.csv uploaded to s3.


In [95]:
import difflib
a = 'WHAT ARE THE ADVANTAGES AND DISADVANTAGES OF STEEL USED AS A STRUCTURAL MATERIAL ?'
b = 'WRITE ADVANTAGES OF STEEL USED AS A STRUCTURAL MATERIAL.'
seq = difflib.SequenceMatcher()
seq.set_seqs(a.lower(), b.lower()) # string b is x times similar to string a 
d = seq.ratio()*100
print(d)

76.81159420289855


In [116]:
from difflib import get_close_matches

word_list =['fjadbbiu', 'jdbfag', 'nsfkubusi', 'bdjfh','gvhvh', 'nfd', 'njfd', a]
matches = get_close_matches('abcd', word_list, n=2, cutoff=0.1)
print(matches)

['bdjfh', 'fjadbbiu']


In [117]:
seq = difflib.SequenceMatcher()
seq.set_seqs('nsfkubusi', 'njfdu') # string b is x times similar to string a 
d = seq.ratio()*100
print(d)

42.857142857142854


In [141]:
from fuzzywuzzy import process

q1_from_s3, similarity = process.extractOne(b, word_list)


In [144]:
similarity

89

In [ ]:
json = {'Records':
        [{'eventVersion': '2.1', 
          'eventSource': 'aws:s3',
          'awsRegion': 'us-west-2',
          'eventTime': '2020-03-14T06:47:35.012Z',
          'eventName': 'ObjectCreated:Put',
          'userIdentity': {'principalId': 'AGR0MSP44PJPT'},
          'requestParameters': {'sourceIPAddress': '223.190.0.32'},
          'responseElements': {'x-amz-request-id': '87F6310717B8F927',
                               'x-amz-id-2': 'P+SdlY01Y1o3nW5GTESwK2Fqeb9U8R4DblU5jFYKjP/fRcCfjT566jWLaewF3bEf7mC6RqSFkzyH41KCXrOvIJiFbxlZ+Uxt'},
          's3': {'s3SchemaVersion': '1.0', 
                 'configurationId': 'c639af7a-bbbf-493a-8c8b-58c3d2e865cd',
                 'bucket': {'name': 'new-qa-test',
                            'ownerIdentity': {'principalId': 'AGR0MSP44PJPT'},
                            'arn': 'arn:aws:s3:::new-qa-test'}, 
                 'object': {'key': 'lambda/qa.csv', 'size': 3207,
                            'eTag': 'c8392caea36e43412cda5a26a52a2792',
                            'sequencer': '005E6C7E076BCBCCD7'}}}]}